In [4]:
%load_ext autoreload
%autoreload 2

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from dask.diagnostics import Profiler, ResourceProfiler, visualize
from datetime import datetime
from satpy import Scene
from glob import glob
import numpy as np
import os

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

from satpy.utils import debug_on
#debug_on()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import dask
dask.config.set(scheduler='threads') 

tod = 'D:/sat_data/sev/out/'


bbox = (-1600000, -2770000, 690000, -1040000)

comp = 'natural_color_flames'
comp = 'fire_radiative_power'

ifiles_l15 = glob('D:/sat_data/sev/in/*.nat')
ifiles_l15.sort()
with dask.config.set({"array.chunk-size": "32MiB"}):
    with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
        st = datetime.utcnow()
        #if not os.path.isfile(outf):
        if True:
            scn = Scene(ifiles_l15, reader='seviri_l1b_native')
            scn.load([comp], generate=False, upper_right_corner='NE')
           # scn.load(['ir_38'], calibration='radiance')
            #scn.load([comp], generate=False)
            scn2 = scn
            scn3 = scn2.resample(scn2.coarsest_area(), resampler='native')
            #scn3.save_dataset(comp, base_dir=tod,  fill_value=0)
            scn3.save_dataset(comp, base_dir=tod, enhance=False, dtype=np.float32, fill_value=0)

        en = datetime.utcnow()
        print('\n', (en-st).total_seconds())
    visualize([prof, rprof], show=False, save=True, filename="D:/sat_data/ahi_main/frp_new.html")

cannot convert float NaN to integer



 27.437312


In [6]:
scn = Scene(ifiles_l15, reader='seviri_l1b_native')
scn.load(['IR_039'], calibration='radiance', upper_right_corner='NE')
scn.save_dataset('IR_039', base_dir=tod, enhance=False, dtype=np.float32, fill_value=0)